In [ ]:
import os

print(os.getcwd())

In [ ]:
import os

# Define the starting directory
start_dir = "../data/fund-holdings"

# Count CSV files
csv_count = sum(
    len(files) for root, _, files in os.walk(start_dir) if any(f.endswith(".csv") for f in files)
)

print(f"Total CSV files found: {csv_count}")


# Determine Unmapped Fund CIK Entries

In [ ]:
import os
import pandas as pd

# Define the starting directory
start_dir = "../data/fund-holdings"

# List to store unique entries without mapped company CIK number
unique_entries = []

# Initialize counter for iteration
file_count = 0

# Iterate through CSV files
for root, _, files in os.walk(start_dir):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, dtype=str)  # Read CSV as string to avoid conversion issues
            
            # Filter rows where mapped_company_cik_number is NaN or empty
            filtered_df = df[df["mapped_company_cik_number"].fillna("").str.strip() == ""]

            # Append unique rows to the list
            unique_entries.extend(filtered_df.drop_duplicates().to_dict(orient="records"))
            
            # Increment file counter and print progress
            file_count += 1
            print(f"Processed {file_count} files...")

# Convert to DataFrame
unique_df = pd.DataFrame(unique_entries)

In [ ]:
unique_df.to_csv("unmapped.csv")

# Determine Most Common (and currently maintained) US-GAAP Columns

In [ ]:
import os
import pandas as pd
from collections import defaultdict

# Define the directory to search for CSV files
start_dir = "../data/orig.us-gaap"

# Dictionary to store column reporting frequency per form type
column_distribution = defaultdict(lambda: {"10-K": 0, "10-Q": 0, "latest_filed": 0})

# Track the number of processed files
file_count = 0

# Define the minimum year threshold
current_year = pd.Timestamp.now().year
min_year = current_year - 4  # Consider only filings within the last 4 years

# Iterate through all CSV files in the directory
for root, _, files in os.walk(start_dir):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)

            try:
                # Read CSV file with dtype=str to prevent automatic type conversion
                df = pd.read_csv(file_path, dtype=str)

                # Ensure required columns exist
                if "form" not in df.columns or "filed" not in df.columns:
                    continue

                # Convert "filed" column to numeric (year only)
                df["filed"] = pd.to_datetime(df["filed"], errors="coerce").dt.year

                # Filter out entries older than min_year
                df = df[df["filed"] >= min_year]

                if df.empty:
                    continue

                # Track the most recent year for each column
                latest_filing_year = df["filed"].max()

                # Count occurrences of each column by form type
                for column in df.columns:
                    form_counts = df["form"].value_counts()
                    for form_type in ["10-K", "10-Q"]:
                        if form_type in form_counts:
                            column_distribution[column][form_type] += form_counts[form_type]

                    # Update latest filing year
                    if latest_filing_year:
                        column_distribution[column]["latest_filed"] = max(
                            column_distribution[column]["latest_filed"], latest_filing_year
                        )

                # Increment processed file counter
                file_count += 1
                if file_count % 10 == 0:
                    print(f"Processed {file_count} files...")

                # TODO: Remove
                # if file_count > 500:
                #     break

            except Exception as e:
                print(f"Error processing {file_path}: {e}")

# Convert results to a DataFrame
df_distribution = pd.DataFrame.from_dict(column_distribution, orient="index")

# Filter columns that haven't been reported in the last 4 years
df_distribution = df_distribution[df_distribution["latest_filed"] >= min_year]

# Sort by most frequently reported in "10-K" and "10-Q"
df_distribution.sort_values(by=["10-K", "10-Q"], ascending=False, inplace=True)

####

# Compute total number of unique 10-K and 10-Q documents (not their sum)
total_10k_docs = df_distribution["10-K"].count()
total_10q_docs = df_distribution["10-Q"].count()

# Compute percentage for each row based on unique document count
df_distribution["10-K %"] = df_distribution["10-K"] / total_10k_docs
df_distribution["10-Q %"] = df_distribution["10-Q"] / total_10q_docs

# Normalize percentages so that the highest value is 100%
df_distribution["10-K %"] = (df_distribution["10-K %"] / df_distribution["10-K %"].max()) * 100
df_distribution["10-Q %"] = (df_distribution["10-Q %"] / df_distribution["10-Q %"].max()) * 100

def classify_statement_type(column_name):
    """Assigns a financial statement type to a given column name using hierarchical classification."""
    best_match = ("Unclassified", 0, None)  # (Statement, Length, Category)

    STATEMENT_TYPES = {
        "Balance Sheet": {
            "Current Assets": [
                "CashAndCashEquivalents", "MarketableSecurities", "AccountsReceivable",
                "Inventory", "PrepaidExpenses", "LoansReceivable", "FairValueMeasurement",
                "CustomerLoyaltyProgram", "FrequentFlierLiability"
            ],
            "Non-Current Assets": [
                "PropertyPlantEquipment", "Goodwill", "IntangibleAssets",
                "DeferredTaxAssets", "LongTermInvestments", "LiabilityForAsbestos",
                "DeferredTaxLiabilities", "AmortizationOfIntangibleAssets"
            ],
            "Liabilities & Equity": [
                "AccountsPayable", "AccruedLiabilities", "DeferredRevenue",
                "ShortTermDebt", "LongTermDebt", "Equity", "RetainedEarnings",
                "OperatingLeaseLiability", "PreferredStock", "UnrecognizedTaxBenefits",
                "AdditionalPaidInCapital"
            ]
        },
        "Income Statement": {
            "Revenue": [
                "Revenue", "Sales", "ServiceRevenue", "InterestAndFeeIncome",
                "SecScheduleSupplementalInformation"
            ],
            "Expenses": [
                "CostOfRevenue", "OperatingExpenses", "InterestExpense", "Depreciation",
                "Amortization", "SellingGeneralAdministrative", "LossOnDisposal",
                "ForeignCurrencyExchangeLoss", "StockBasedCompensationExpense",
                "AdjustmentsForIncomeTaxExpense", "AdjustmentsForAmortizationExpense",
                "OtherFinanceIncomeCost", "OperatingLeaseCost"
            ],
            "Profit & Loss": [
                "GrossProfit", "NetIncome", "Loss", "ComprehensiveIncome",
                "ComprehensiveIncomeNetOfTax", "ProfitLoss"
            ],
            "Cash-Related Adjustments": [
                "InterestIncome", "InterestExpense", "Depreciation",
                "Amortization", "StockBasedCompensation"
            ]
        },
        "Cash Flow Statement": {
            "Operating Activities": [
                "CashFlowsFromOperatingActivities", "IncreaseInInventory", "Depreciation",
                "StockBasedCompensation", "AccountsReceivableChanges", "OperatingActivities",
                "NoncashAcquisition", "ReorganizationValueCash", "OperatingLeasePayments"
            ],
            "Investing Activities": [
                "CapitalExpenditures", "Investments", "PurchaseOfProperty", "SaleOfInvestments",
                "ProceedsFromIssuanceOfSecurities", "PaymentsForInvestment"
            ],
            "Financing Activities": [
                "DebtIssuance", "StockRepurchase", "DividendsPaid", "ProceedsFromStockOptions",
                "RepaymentOfLongTermDebt", "LiabilitiesArisingFromFinancingActivities",
                "PaymentsForRepurchaseOfStock"
            ],
            "Other Cash Flow Items": [
                "ForeignCurrencyExchangeEffects", "InterestPaid", "IncomeTaxesPaid",
                "NonCashItems", "ChangesInWorkingCapital", "PaymentsForRestrictedCash"
            ]
        }
    }

    for statement, categories in STATEMENT_TYPES.items():
        for category, keywords in categories.items():
            for keyword in sorted(keywords, key=len, reverse=True):  # Prioritize longest match
                if keyword in column_name and len(keyword) > best_match[1]:
                    best_match = (statement, len(keyword), category)

    return best_match[0], best_match[2]  # Returns (Statement Type, Category)


# Apply classification to each column
df_distribution["Statement Type"] = df_distribution.index.map(classify_statement_type)

In [ ]:
df_distribution.to_csv("column_distribution.csv")

In [ ]:
import re
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# Load FinBERT model and tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Extended category structure (4 levels deep)
STATEMENT_TYPES = {
    "Balance Sheet": {
        "Assets": {
            "Cash and Short Term Assets": ["Cash", "Cash & Equivalents", "Short Term Investments"],
            "Accounts Receivable - Trade, Net": ["Accounts Receivable - Trade, Gross"],
            "Total Receivables, Net": ["Receivables, Other"],
            "Total Inventory": ["Inventories - Finished Goods", "Inventories - Work in Progress"],
            "Prepaid Expenses": [],
            "Other Currrent Assets, Total": ["Restricted Cash - Current", "Other Current Assets"]
        },
        "Property/Plant/Equipment, Total - Gross": ["Buildings - Gross", "Land & Improvements - Gross", "Machinery & Equipment - Gross", "Other Property/Plant/Equipment - Gross"],
        # "Liabilities & Equity": {
        #     "Current Liabilities": ["Accounts Payable", "Short Term Debt"],
        #     "Non-Current Liabilities": ["Long Term Debt", "Deferred Tax Liabilities"]
        # }
    },
    # "Income Statement": {
    #     "Revenue": ["Sales", "Service Revenue", "Interest Income"],
    #     "Expenses": {
    #         "Cost of Revenue": ["Direct Costs", "COGS"],
    #         "Operating Expenses": ["Selling, General and Administrative", "R&D"]
    #     },
    #     "Profit & Loss": ["Gross Profit", "Net Income"]
    # },
    # "Cash Flow Statement": {
    #     "Operating Activities": ["Cash Received from Customers", "Interest Paid"],
    #     "Investing Activities": ["Capital Expenditures", "Investments"],
    #     "Financing Activities": ["Debt Issuance", "Stock Repurchase"]
    # }
}

# Function to get embeddings from FinBERT
def get_embedding(text):
    """Generates an embedding for the given text using FinBERT."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # Use CLS token embedding

# Prepare category embeddings (combine terms for each category)
def prepare_category_embeddings():
    match_terms = []
    for statement, categories in STATEMENT_TYPES.items():
        for category, subcategories in categories.items():
            # TODO: I'm not sure if including statement and category is beneficial or is deteriorating the performance
            if isinstance(subcategories, dict):  # If it has further subcategories
                for subcategory, terms in subcategories.items():
                    # Combine the category and subcategory with the terms
                    combined_terms = f"{statement} {category} {subcategory} " + " ".join(terms).lower()
                    match_terms.append((combined_terms, statement, category, subcategory))
            else:
                # If no subcategories, just combine the category name and terms
                combined_terms = f"{statement} {category} " + " ".join(subcategories).lower()
                match_terms.append((combined_terms, statement, category, ""))
    
    # Print to verify
    print(match_terms)
    
    # Create DataFrame with all combined terms
    match_df = pd.DataFrame(match_terms, columns=["Processed Terms", "Statement", "Main Category", "Subcategory"])
    
    # Generate embeddings for each combined category of terms
    match_df["Embedding"] = match_df["Processed Terms"].apply(lambda x: get_embedding(x))
    
    return match_df


# Function to classify US-GAAP field names using FinBERT embeddings
def classify_us_gaap_field(field_name, match_df, top_n=3):
    """Classifies a US-GAAP field using FinBERT embeddings."""
    # Preprocess input (split camel case)
    processed_field = re.sub(r'(?<!^)(?=[A-Z])', ' ', field_name).lower()

    print(processed_field)
    
    # Generate embedding for input field
    field_embedding = get_embedding(processed_field)
    
    # Compute similarity scores
    similarities = [F.cosine_similarity(field_embedding, emb).item() for emb in match_df["Embedding"]]
    
    # Rank and return top matches
    match_df["Similarity"] = similarities
    return match_df.sort_values(by="Similarity", ascending=False).head(top_n)

# Example usage
match_df = prepare_category_embeddings()  # Precompute category embeddings
us_gaap_field = "PropertyPlantEquipment"
top_matches = classify_us_gaap_field(us_gaap_field, match_df)

# Display the results
top_matches


# US-GAAP 2025 Hierarchy

https://www.sec.gov/data-research/standard-taxonomies/operating-companies